# Código em atualização.

In [ ]:
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx2pdf import convert
import time
from num2words import num2words
import re
import os
from unidecode import unidecode

class BuscarDados:

    def __init__(self, busca, aba=None, arquivo='01-Cadastro.xlsm'):
        
        banco_de_dados = (
            pd.read_excel(arquivo, aba, dtype=str) if aba
            else pd.concat(pd.read_excel(arquivo, aba, dtype=str), ignore_index=True)
        )
        
        banco_de_dados = banco_de_dados.applymap(lambda texto: texto.lower() if isinstance(texto, str) else texto)
        busca = str(busca).lower()

        try:
            self.dicionario = banco_de_dados[banco_de_dados.eq(busca).any(axis=1)].iloc[0].dropna().to_dict()
        except IndexError:
            raise ValueError('O valor não foi encontrado nas abas do arquivo')
        
        self.dicionario = {chave.lower(): valor for chave, valor in self.dicionario.items()}

class FormatarTexto:
    
    def __init__(self, dicionario):
        
        maiusculas = 'nome','uf','expedidor_rg','administrador','comarca_uf'
        minusculas = 'de','com','da','das','do','dos','na','nas','no','nos','o','os','a','as','uma','umas','um','uns'
        inicial_maiuscula = 'logradouro','complemento','bairro','cidade','vara','comarca'

        for chave, valor in dicionario.items():
            
            if chave in maiusculas:
                dicionario[chave] = valor.upper()
            
            elif chave in inicial_maiuscula:
                palavras = valor.split()
                palavras_atualizadas = (
                    palavra.title() if palavra.lower() not in minusculas else palavra
                    for palavra in palavras
                )
                dicionario[chave] = ' '.join(palavras_atualizadas)

class AgruparEndereco:
    def __init__(self, dicionario):
        
        try:
            n = f"{dicionario['numero']} - {dicionario ['complemento']}" if 'complemento' in dicionario else dicionario['numero']
            endereco = f"{dicionario['logradouro']}, nº {n}, Bairro {dicionario['bairro']}, CEP {dicionario['cep']}, {dicionario['cidade']}/{dicionario['uf']}"
            dicionario['endereco'] = endereco
        except KeyError as e:
            raise ValueError(f'O dicionário não possui a chave {e.args[0]} para construir o endereço')
        
class QualificarParte:
    
    def __init__(self, dicionario):
        
        if 'cpf' in dicionario:
            qualificacao = (
                f"{dicionario['nome']}, {dicionario['nacionalidade']}, {dicionario['estado_civil']}, {dicionario['profissao']}, RG n° {dicionario['rg']} "
                f"{dicionario['expedidor_rg']}, CPF n° {dicionario['cpf']}, e-mail: {dicionario['email']}, com endereço à {dicionario['endereco']}"
            )

        elif 'tag' in dicionario and dicionario['tag'] == 'condominio':
            qualificacao = (
                f"{dicionario['nome']}, inscrito no CNPJ n° {dicionario['cnpj']}, com endereço à {dicionario['endereco']}, "
                f"devidamente representado neste ato pelo seu síndico(a) {dicionario['administrador']}"
            )
        
        else:
            qualificacao = (
                f"{dicionario['nome']}, pessoa jurídica inscrita no CNPJ n° {dicionario['cnpj']}, com endereço à {dicionario['endereco']},"
                f" devidamente representada neste ato pelo(a) sócio(a) {dicionario['administrador']}"
            )
        
        dicionario['qualificacao'] = qualificacao

class DividirValor:
    
    def __init__(self, dicionario, separador):
        dicionario.update({chave: valor.split(separador) for chave, valor in dicionario.items() if separador in valor})

class ParearAtributos:
    
    def __init__(self, dicionario, par_1, par_2):
        
        CriarLista(dicionario, par_1)
        CriarLista(dicionario, par_2)
        dicionario.update(dict(zip(dicionario[par_1], dicionario[par_2])))

class CriarLista:
    
    def __init__(self, dicionario, valor):
        if valor in dicionario:
            dicionario[valor] = [dicionario[valor]] if isinstance(dicionario[valor], str) else dicionario[valor]

class CriarPessoa:

    def __init__(self, dicionario):
        
        FormatarTexto(dicionario)
        AgruparEndereco(dicionario)
        QualificarParte(dicionario)

class CriarPartes:
    
    def __init__(self, dicionario):
        
        for parte in dicionario['partes']:
            for i, pessoa in enumerate(dicionario[parte], start=1):
                prefixo = f'[{i}]{parte}_'
                
                dicionario_pessoa = BuscarDados(pessoa).dicionario
                CriarPessoa(dicionario_pessoa)

                for chave, valor in dicionario_pessoa.items():
                    dicionario[prefixo + chave] = valor

                if len(dicionario['partes']) == 1:
                    dicionario[parte + '_qualificacao'] = dicionario[f'{prefixo}qualificacao']                    
                elif i == 1:
                    dicionario[parte + '_qualificacao'] = f'[{i}] ' + dicionario[f'{prefixo}qualificacao']
                elif i == len(dicionario['partes']):
                    dicionario[parte + '_qualificacao'] += ' e ' + f'[{i}] ' + dicionario[f'{prefixo}qualificacao']
                else:
                    dicionario[parte + '_qualificacao'] += ', ' + f'[{i}] ' + dicionario[f'{prefixo}qualificacao']

class CriarNomeArquivo:

     def __init__(self, dicionario):      
        
        peca = dicionario['peca'] + '_'

        if isinstance(dicionario['partes'], str):
            parte = dicionario['pessoas']        
        else:
            parte = dicionario['pessoas'][0]
        
        dicionario['arquivo'] = unidecode((peca + parte).replace(' ','-'))

class CriarDocumento:

    def __init__(self, id_documento):

        self.dicionario = BuscarDados(id_documento, 'Peticoes').dicionario
        self.dicionario.update(BuscarDados(self.dicionario['peca'], 'Modelos').dicionario)
        
        DividirValor(self.dicionario, ';')
        if 'atributos' in self.dicionario:
            ParearAtributos(self.dicionario, 'atributos', 'valores')
        ParearAtributos(self.dicionario, 'partes', 'pessoas')
        
        DividirValor(self.dicionario, '/')        

        for parte in self.dicionario['partes']:
            CriarLista(self.dicionario, parte)
        for pessoa in self.dicionario['pessoas']:
            CriarLista(self.dicionario, pessoa)
        
        adicionar_dados_do_processo(self.dicionario)
        CriarPartes(self.dicionario)
        CriarNomeArquivo(self.dicionario)

class SalvarDocxPdf:

    def __init__(self, id_documento):
        
        dicionario = CriarDocumento(id_documento).dicionario
        documento = Document(f"01_Modelos\{dicionario['peca']}.docx")
        self._substituir_texto(dicionario, documento)      
        
        documento.save(f"02_Documentos-prontos\{dicionario['arquivo']}.docx")
        convert(f"02_Documentos-prontos\{dicionario['arquivo']}.docx", f"02_Documentos-prontos\{dicionario['arquivo']}.pdf")
        
    def _substituir_texto(self, dicionario, docx):

        def numero_por_extenso(paragrafo):
            
            numero_entre_parenteses = r"\((.*?)\)"
            resultado_busca_numero = re.search(numero_entre_parenteses, paragrafo.text)
            if resultado_busca_numero:
                valor_encontrado = resultado_busca_numero.group(1)
                valor_decimal = valor_encontrado.replace('.', '').replace(',', '.')
                try:
                    valor_por_extenso = '(' + num2words(float(valor_decimal), lang='pt_BR', to='currency') + ')'
                    paragrafo.text = paragrafo.text.replace(resultado_busca_numero.group(0), valor_por_extenso)
                except ValueError:
                    pass

        dicionario_de_substituicao = {'x' + chave.upper() + 'x': str(valor) for chave, valor in dicionario.items()}
        
        for paragrafo in docx.paragraphs:
            for chave in dicionario_de_substituicao:

                if not isinstance(chave, list):
                    if chave in paragrafo.text:
                        paragrafo.text = paragrafo.text.replace(chave, dicionario_de_substituicao[chave])
                numero_por_extenso(paragrafo)

